# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [2]:
# Your code here; import the necessary packages

import pandas as pd
import sqlite3



In [4]:
# Your code here; create the connection

conn = sqlite3.connect("data.sqlite")

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [19]:
# Your code here

query = """

SELECT
    customers.customerNumber,
    customers.contactLastName,
    customers.contactFirstName
    
  FROM customers
    
WHERE customerNumber IN (SELECT customerNumber
                           FROM   orders
                           WHERE orderDate = '2003-01-31') 

;

"""

pd.read_sql(query,conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [39]:
# Your code here

query = """

  SELECT productName, total_orders
  FROM products
  JOIN
  (SELECT productCode, 
          SUM (quantityOrdered) AS total_orders
          FROM orderdetails
          GROUP BY productCode) AS order_totals
  
  ON products.productCode = order_totals.productCode
  
  ORDER BY total_orders;
  
"""

pd.read_sql(query,conn)

,productName,total_orders
0,1957 Ford Thunderbird,767
1,1970 Chevy Chevelle SS 454,803
2,1936 Mercedes Benz 500k Roadster,824
3,1911 Ford Town Car,832
4,1999 Indy 500 Monte Carlo SS,855
...,...,...
104,1930 Buick Marquette Phaeton,1074
105,1941 Chevrolet Special Deluxe Cabriolet,1076
106,American Airlines: MD-11S,1085
107,1937 Lincoln Berline,1111


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [66]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT


"""
Breakdown:

1. We need to get the product names associated with each order using the orderNumber - INNER QUERY
2. WE need to get distinct customer number associated with Product Name  and group by product name and order 
   by no of customers - OUTER QUERY

"""

query = """

SELECT orders_and_products.productName, COUNT( DISTINCT orders.customerNumber) AS no_of_customers
 
 FROM  (SELECT products.productName, orderdetails.orderNumber
        FROM products
             JOIN orderdetails
             ON orderdetails.productCode = products.productCode) AS orders_and_products
             
 JOIN   orders
   ON   orders.orderNumber = orders_and_products.orderNumber
   GROUP BY orders_and_products.productName
   ORDER BY  no_of_customers DESC;
             
"""
                             
                                                     

pd.read_sql(query,conn)

,productName,no_of_customers
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [81]:
# Your code here

"""
1. We need to get the employee numbers, first names , lastnames and city(of office) and office code
2. We need to get the customer numbers associated with each employee
3. We need to get order numbers for each customer

"""

query = """

SELECT  employee_customer_details.employeeNumber, 
        employee_customer_details.firstName, 
        employee_customer_details.lastName,
        employee_customer_details.city,
        employee_customer_details.officeCode,
        employee_customer_details.salesRepEmployeeNumber,
        count(orders.customerNumber) AS no_of_orders

FROM    (SELECT employee_details.employeeNumber, 
                employee_details.firstName, 
                employee_details.lastName,
                employee_details.city,
                employee_details.officeCode,
                customers.salesRepEmployeeNumber,
                customers.customerNumber
       

           FROM  (SELECT employees.employeeNumber, 
                         employees.firstName, 
                         employees.lastName, 
                         offices.city, 
                         offices.officeCode
                    FROM employees
                         JOIN offices
                         ON employees.officeCode = offices.officeCode) AS employee_details
                
         JOIN customers
          ON customers.salesRepEmployeeNumber = employee_details.employeeNumber) AS employee_customer_details

JOIN orders
  ON orders.customerNumber = employee_customer_details.customerNumber
  
  GROUP BY employee_customer_details.employeeNumber
  HAVING no_of_orders < 20 ;
          

"""


pd.read_sql(query,conn)


,employeeNumber,firstName,lastName,city,officeCode,salesRepEmployeeNumber,no_of_orders
0,1166,Leslie,Thompson,San Francisco,1,1166,14
1,1188,Julie,Firrelli,Boston,2,1188,14
2,1216,Steve,Patterson,Boston,2,1216,18
3,1286,Foon Yue,Tseng,NYC,3,1286,17
4,1611,Andy,Fixter,Sydney,6,1611,19
5,1612,Peter,Marsh,Sydney,6,1612,19
6,1621,Mami,Nishi,Tokyo,5,1621,16
7,1702,Martin,Gerard,Paris,4,1702,12


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [103]:
# Your code here


query = """

SELECT customer_employees.employeeNumber,
       customer_employees.firstName,
       customer_employees.lastName,
       AVG(customer_employees.creditLimit) AS average_customer_credit_limit
       
FROM   (SELECT employees.employeeNumber, 
               employees.firstName, 
               employees.lastName, 
               customers.customerNumber,
               customers.creditLimit
           FROM employees
                JOIN customers
                ON employees.employeeNumber = customers.salesRepEmployeeNumber) AS customer_employees

GROUP BY customer_employees.employeeNumber

HAVING average_customer_credit_limit > 15000
;
"""

pd.read_sql(query,conn)


,employeeNumber,firstName,lastName,average_customer_credit_limit
0,1165,Leslie,Jennings,100433.333333
1,1166,Leslie,Thompson,65266.666667
2,1188,Julie,Firrelli,73916.666667
3,1216,Steve,Patterson,81533.333333
4,1286,Foon Yue,Tseng,66614.285714
5,1323,George,Vanauf,80887.500000
6,1337,Loui,Bondur,86233.333333
7,1370,Gerard,Hernandez,91785.714286
8,1401,Pamela,Castillo,81340.000000
9,1501,Larry,Bott,91187.500000


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!